<a href="https://colab.research.google.com/github/Egoluback/aijic2021_transport/blob/yarik/model4track.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import time
import seaborn as sns
import matplotlib.pyplot as plt
from imblearn.under_sampling import RandomUnderSampler

/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


In [2]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [3]:
path = './drive/MyDrive/aiijc_transport_simpleteam/'

train_labeled = pd.read_csv(path + 'data/base_files/labled_train_data.csv', index_col=0, sep='\t', comment='#')
comments_labeled = pd.read_csv(path + 'data/base_files/labled_train_comments.csv', index_col=0, sep='\t', comment='#')
tracks_labeled = pd.read_csv(path + 'data/labled_train_tracks_speed.csv', index_col=0, sep=',', comment='#')

train_unlabeled = pd.read_csv(path +'data/base_files/unlabled_train_data.csv', index_col=0, sep='\t', comment='#')
comments_unlabeled = pd.read_csv(path+ 'data/base_files/unlabled_train_comments.csv', index_col=0, sep='\t', comment='#')
tracks_unlabeled = pd.read_csv(path+ 'data/unlabled_train_tracks_speed.csv', index_col=0, sep='\t', comment='#')

In [16]:
tracks_labeled

,Unnamed: 0.1,driver_id,dt,lat_,lon_,order_id,speed,is_aggressive
0,0,b76545fa3cc14acd6a69ac13c1edac33,2021-02-09 21:43:41,55.792710,37.545409,001662da857b5a39bb402aacf3145f86,NaN,0.0
1,1,b76545fa3cc14acd6a69ac13c1edac33,2021-02-09 21:44:40,55.792013,37.544481,001662da857b5a39bb402aacf3145f86,5.906441,0.0
2,2,b76545fa3cc14acd6a69ac13c1edac33,2021-02-09 21:45:00,55.791365,37.543695,001662da857b5a39bb402aacf3145f86,15.696000,0.0
3,3,b76545fa3cc14acd6a69ac13c1edac33,2021-02-09 21:45:20,55.791267,37.543512,001662da857b5a39bb402aacf3145f86,2.862000,0.0
4,4,b76545fa3cc14acd6a69ac13c1edac33,2021-02-09 21:45:37,55.791175,37.543634,001662da857b5a39bb402aacf3145f86,2.710588,0.0
...,...,...,...,...,...,...,...,...
675217,675217,1c11e4febcc32f17e7fbbd20a86b9a59,2021-03-04 19:38:08,55.833583,37.490335,fffdfda358f35cf2d7f9d87d205655a7,36.891429,0.0
675219,675219,1c11e4febcc32f17e7fbbd20a86b9a59,2021-03-04 19:38:57,55.833093,37.491756,fffdfda358f35cf2d7f9d87d205655a7,8.517273,0.0
675220,675220,1c11e4febcc32f17e7fbbd20a86b9a59,2021-03-04 19:39:14,55.832650,37.493704,fffdfda358f35cf2d7f9d87d205655a7,27.783529,0.0
675222,675222,1c11e4febcc32f17e7fbbd20a86b9a59,2021-03-04 19:39:37,55.833031,37.495477,fffdfda358f35cf2d7f9d87d205655a7,23.720000,0.0


In [ ]:
tracks_unlabeled['is_aggressive']=-1
all_tracks = pd.concat([tracks_labeled, tracks_unlabeled])
all_tracks

,Unnamed: 0.1,driver_id,dt,lat_,lon_,order_id,speed,is_aggressive
0,0.0,b76545fa3cc14acd6a69ac13c1edac33,2021-02-09 21:43:41,55.792710,37.545409,001662da857b5a39bb402aacf3145f86,NaN,0.0
1,1.0,b76545fa3cc14acd6a69ac13c1edac33,2021-02-09 21:44:40,55.792013,37.544481,001662da857b5a39bb402aacf3145f86,5.906441,0.0
2,2.0,b76545fa3cc14acd6a69ac13c1edac33,2021-02-09 21:45:00,55.791365,37.543695,001662da857b5a39bb402aacf3145f86,15.696000,0.0
3,3.0,b76545fa3cc14acd6a69ac13c1edac33,2021-02-09 21:45:20,55.791267,37.543512,001662da857b5a39bb402aacf3145f86,2.862000,0.0
4,4.0,b76545fa3cc14acd6a69ac13c1edac33,2021-02-09 21:45:37,55.791175,37.543634,001662da857b5a39bb402aacf3145f86,2.710588,0.0
...,...,...,...,...,...,...,...,...
"674713,cb82836394c8d6b49258c2993d833587,2021-03-18 23:45:53,55.7936508,37.4938731,b8af43aa9f542bb6d869b19690a8ebbd,0.26441944973178755",NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.0
"674714,6354ef9186ef028b633d598354b190d7,2021-03-18 19:26:55,55.7488301,37.5761351,c6f2ab6dbacc1655909281124a7db5db,-1.6596421675891364",NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.0
"674715,4a43794a11c47f3f62eef22b97f6f9b1,2021-03-18 08:30:09,55.9104284,37.5840968,56c1826d6060e1bd19125b0cdaebb781,-1.6422088006902502",NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.0
"674716,df80d194cbb12e286c8b5175d9938dfe,2021-03-18 23:56:01,55.6566868,37.846926,ca267eb622d411adb713703d738ff98f,2.115978542626728",NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.0


each order is different batch

In [16]:
def preprocess(tracks, chunk_size, multiplier):
  
  # undersampling method deletes some extra non aggressive values
  def undersampling(X, multiplier):
    aggressive_count = sum(X.is_aggressive==1)
    non_aggressive_ind = X[X.is_aggressive==0].index

    # number of aggressive and non-aggressive labels is the same
    random_indices = np.random.choice(non_aggressive_ind, aggressive_count*multiplier, replace=False)
    return X.loc[random_indices]

  def split(arr, chunk_size = 15):
    result = []
    #get right length of arr so that it equally splits into chunks
    length = len(arr)
    split_length = length - (length%chunk_size)
            
    for i in range(split_length)[chunk_size::chunk_size]:
        result.append(arr[i-chunk_size:i])

    return np.array(result)

  # make df, so that each row has whole order speeds time series
  def make_nested(tracks, chunk_size, multiplier):
    unique_orders= tracks.drop_duplicates('order_id', keep='last')
    unique_orders = undersampling(unique_orders, multiplier)
    y_labels = []
    X_train = []
    for order in unique_orders['order_id']:
      order_df = tracks[tracks.order_id==order]
      order_df.loc[0, 'speed']=0

      if order_df.shape[0]<chunk_size:
        continue

      splitted_arrs = split(order_df.values, chunk_size)
      for arr in splitted_arrs:
        is_aggressive = arr[0][7]
        y_labels.append(is_aggressive)
        speed_series = []
        for row in arr:  
            # append only speed and dt values 
            speed_series.append(row[6])
        X_train.append(pd.Series(speed_series))
    return X_train, y_labels

  X_train, train_labels = make_nested(tracks, chunk_size, multiplier)

  X_train = pd.DataFrame({'speed':X_train})
  y_train = np.array(train_labels)

  return X_train, y_train

CHUNK_SIZE = 20 
MULTIPLIER = 3 
X_train, y_train = preprocess(tracks_labeled, CHUNK_SIZE, MULTIPLIER)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [17]:
X_train

,speed
0,0 -0.113554 1 3.562759 2 26.92800...
1,0 0.008005 1 2.755385 2 0.35122...
2,0 -0.006301 1 0.892800 2 0.28636...
3,0 6.300000 1 2.491200 2 4.66000...
4,0 0.050996 1 3.960000 2 4.69...
...,...
1439,0 83.595789 1 72.480000 2 79.35...
1440,0 -0.024098 1 34.200000 2 7.86857...
1441,0 38.926957 1 28.722857 2 32.86285...
1442,0 13.221818 1 7.046809 2 27.08341...


In [18]:
from sklearn.model_selection import KFold  
from sklearn.pipeline import make_pipeline
from sktime.transformations.panel.rocket import MiniRocket
from sklearn.model_selection import train_test_split
from sklearn.linear_model import RidgeClassifierCV
from sklearn.semi_supervised import SelfTrainingClassifier

X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.33, random_state=42)
rocket = MiniRocket()
rocket.fit(X_train, y_train)
X_train_transform = rocket.transform(X_train,y_train)

classifier = RidgeClassifierCV(alphas = np.logspace(-3, 3, 10), normalize = True)
#classifier = SelfTrainingClassifier(classifier)
classifier.fit(X_train_transform, y_train)

X_test_transform = rocket.transform(X_test)
classifier.score(X_test_transform, y_test)
classifier


RidgeClassifierCV(alphas=array([1.00000000e-03, 4.64158883e-03, 2.15443469e-02, 1.00000000e-01,
       4.64158883e-01, 2.15443469e+00, 1.00000000e+01, 4.64158883e+01,
       2.15443469e+02, 1.00000000e+03]),
                  class_weight=None, cv=None, fit_intercept=True,
                  normalize=True, scoring=None, store_cv_values=False)

In [7]:
!pip install sktime

     |████████████████████████████████| 5.8 MB 6.8 MB/s 
     |████████████████████████████████| 22.3 MB 1.4 MB/s 
     |████████████████████████████████| 9.5 MB 66.6 MB/s 
  Attempting uninstall: statsmodels
    Found existing installation: statsmodels 0.10.2
    Uninstalling statsmodels-0.10.2:
      Successfully uninstalled statsmodels-0.10.2
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 0.22.2.post1
    Uninstalling scikit-learn-0.22.2.post1:
      Successfully uninstalled scikit-learn-0.22.2.post1


In [19]:
classifier.score(X_test_transform, y_test)

1.0